# Running target selection and working with desitarget#
### Author: Adam D. Myers, University of Wyoming ###

This Notebook describes how to produce Target Selection files for a Data Release, focusing on Data Release 7 (DR7) of the Legacy Surveys. In addition, the final section contains tips and tricks for working with the code.

# Setting up your environment #

First, ensure that your environment matches a standard DESI environment. For example:

```
module unload desimodules
source /project/projectdirs/desi/software/desi_environment.sh 18.7
```

`desitarget` relies on `desiutil` and `desimodel`, so you may also need to set up a wider DESI environment, as detailed at:

https://desi.lbl.gov/trac/wiki/Pipeline/GettingStarted/Laptop/JuneMeeting

It may also be useful to set up some additional environment variables that are used in some versions of the `desitarget` code (you could also place these in your `.bash_profile.ext` file):

```
export DESIMODEL=$HOME/git/desimodel
export DUST_DIR=/project/projectdirs/desi/software/edison/dust/v0_1/maps
export GAIA_DIR=/project/projectdirs/desi/target/gaia_dr2
```

Here, I've set `DESIMODEL` to a *reasonable* location. For a more detailed description of checking out the `desimodel` data files from svn see:

https://desi.lbl.gov/trac/wiki/Pipeline/GettingStarted/Laptop/JuneMeeting#Datafilesfordesimodel

# Basic production of target files at the command line#

Here is an example of running an entire DR7 release using file paths at NERSC and a tagged version of the `desitarget` code (`0.22.0` for this example).

First set up the needed directories and a version number:

```
export LSDIR='/global/project/projectdirs/cosmo/data/legacysurvey/dr7'
export TARGDIR='/project/projectdirs/desi/target/catalogs/dr7.1/0.22.0'
export WEBDIR='/project/projectdirs/desi/www/users/adamyers'
export DR='7.1'
export VERSION='0.22.0'
```

Now swap the currently loaded version of `desitarget` for the tagged version:

```
module swap desitarget/$VERSION
```

Make the directory for output targets, if it doesn't exist:

```
mkdir $TARGDIR
```

From this point forward, we will be running production-level code at NERSC. You may want to familiarize yourself with how to run batch jobs at NERSC. A straightforward way to run the code is to request interactive nodes. For example, to reserve one interactive node for 5 minutes:

```
salloc -N 1 -C haswell -t 00:05:00 --qos interactive -L SCRATCH,project
```

Now, to produce the file of targets:

```
select_targets $LSDIR/sweep/$DR/ $TARGDIR/targets-dr$DR-$VERSION.fits
```
(this takes a little over an hour on 1 Cori node, running with the default 32 processes per node).

You will now have produced an output targets file in `$TARGDIR/targets-dr$DR-$VERSION.fits`.

You should remember to change permissions on the file if other people will use it, e.g.:

```
chgrp desi $TARGDIR/targets-dr$DR-$VERSION.fits ; chmod 660 $TARGDIR/targets-dr$DR-$VERSION.fits
```

# Production of all desitarget files at the command line #


In addition to the basic targets file, you may want some of the other `desitarget` products.

To produce a random catalog:

```
select_randoms $LSDIR $CSCRATCH/blat.fits --bundlebricks 20000
```
Then, try a few different numbers in place of 20000 until you produce a script that 
is balanced across nodes. Once you have a balanced script, follow the onscreen instructions 
and move the output file to `$TARGDIR/randoms-dr$DR-$VERSION.fits`. This takes about 1.5-2.5 
hours to run on about 10 Cori nodes with 32 processes per node.

To produce a HEALPixel map of weights from a random catalog:

```
make_imaging_weight_map $TARGDIR/randoms-dr$DR-$VERSION.fits $TARGDIR/targets-dr$DR-$VERSION.fits $TARGDIR/pixweight-dr$DR-$VERSION.fits
```
This takes about 20 minutes to run on one Cori node with 1 process per node.

To use a HEALPixel weight file, a random catalog, and a target catalog to produce a set 
of QA webpages (e.g. see [the DR7 QA pages]:

[the DR7 QA pages]: http://portal.nersc.gov/project/desi/users/adamyers/desitargetQA-dr7.1-0.22.0/

```  
run_target_qa $TARGDIR/targets-dr$DR-$VERSION.fits $WEBDIR/desitargetQA-dr$DR-$VERSION -w $TARGDIR/pixweight-dr$DR-$VERSION.fits
```  
This takes about 1.5 hours on one Cori node with 1 process per node.

To select sky targets for a data release:

```
select_skies $LSDIR $CSCRATCH/blat.fits --bundlebricks 10000
```
Then, try a few different numbers in place of 10000 until you produce a script that 
is balanced across nodes. Once you have a balanced script, follow the onscreen instructions 
and move the output file to `$TARGDIR/skies-dr$DR-$VERSION.fits`). This takes about 3-4
hours to run on about 15 Cori nodes with 32 processes per node.

Finally, to produce a set of GFA targets:

```
select_gfas $LSDIR/sweep/$DR $TARGDIR/gfas-dr$DR-$VERSION.fits
```  
This takes about 50 minutes to run on one Cori node with 32 processes per node.


Again, remember to change permissions on the file if other people will use it, e.g.:

```
chgrp desi *fits ; chmod 660 *fits
```



# Tips and tricks for working with command-line desitarget code #

## General

- All desitarget executables listed in the previous section have a `-h` flag that will 
describe how to use that code, e.g.:

```
select_targets -h
```

## select_targets


- It is possible to select just one target class. For example, to select (set the targeting bits for) 
only the quasar target class:

```
select_targets $LSDIR/sweep/$DR/ $TARGDIR/targets-dr$DR-$VERSION.fits --tcnames QSO
```
the ``tcnames`` flag should be interpreted as "target class names."

- Multiple target classeses can also be selected by passing a comma-separated list. For example, to 
select only the LRG and ELG target classes:

```
select_targets $LSDIR/sweep/$DR/ $TARGDIR/targets-dr$DR-$VERSION.fits --tcnames LRG,ELG
```

- Versions of `select_targets` also exist for commissioning (cmx) and Survey Validation (SV). An example of running the commissioning version of the code is:

```
select_cmx_targets $LSDIR/sweep/$DR/ $TARGDIR/cmx-targets-dr$DR-$VERSION.fits
```

The commissioning code runs very fast, so has no options beyond the input directory and output file.

An example of running the SV version of the code is:

```
select_sv_targets $LSDIR/sweep/$DR/ $TARGDIR/sv2-targets-dr$DR-$VERSION.fits --iteration 2
```

The SV code has many of the same options as `select_targets` but has the additional flag `--iteration` to denote *which* version of SV is being run (e.g., the first iteration of SV corresponds to `-- iteration 1`).

## run_target_qa

- A similar trick to the use of `tcnames` in the previous section can be used to produce web pages 
for a subset of targets. For example:

```
run_target_qa $TARGDIR/targets-dr$DR-$VERSION.fits $WEBDIR/desitargetQA-dr$DR-$VERSION -w $TARGDIR/pixweight-dr$DR-$VERSION.fits --bitnames LRG,ELG
```
(note that eventually `--bitnames` may be renamed `--tcnames` for consistency).

- It isn't necessary to use a weight map if you don't have an appropriate one made. Just drop the `-w` flag, e.g.:

```
run_target_qa $TARGDIR/targets-dr$DR-$VERSION.fits $WEBDIR/desitargetQA-dr$DR-$VERSION
```

Without the map, the code will produce the same output but with equal weighting for all areas of the imaging footprint.

- Producing plots is the slowest part of running targeting QA. To produce the webpages without making any new plots:

```
run_target_qa $TARGDIR/targets-dr$DR-$VERSION.fits $WEBDIR/desitargetQA-dr$DR-$VERSION --noplots
```

- If a weight map is passed, `run_target_qa` produces plots of systematics across the imaging footprint by default. It can take some time to produce these plots, but they can be turned off with the --nosystematics flag:

```
run_target_qa $TARGDIR/targets-dr$DR-$VERSION.fits $WEBDIR/desitargetQA-dr$DR-$VERSION -w $TARGDIR/pixweight-dr$DR-$VERSION.fits --nosystematics.
```



# Directly calling desitarget selection functions#

The main module that performs the selection work for desitarget is called cuts. It has many useful functions for target selection "under the hood" that could be called directly from Python. For example the command line call:

```
export LSDIR='/global/project/projectdirs/cosmo/data/legacysurvey/dr7'
export TARGDIR='/project/projectdirs/desi/target/catalogs/dr7.1/0.22.0'
export DR='7.1'
export VERSION='0.22.0'
```

```
select_targets $LSDIR/sweep/$DR/ $TARGDIR/targets-dr$DR-$VERSION.fits --tcnames LRG,ELG
```

Is equivalent to the following set of commands at the Python prompt:

In [ ]:
from desitarget.cuts import select_targets
from desitarget import io

# ADM inputs
nside = io.desitarget_nside()
src = '/global/project/projectdirs/cosmo/data/legacysurvey/dr7/sweep/7.1'
dest = '/project/projectdirs/desi/target/catalogs/dr7.1/0.22.0/targets-dr7.1-0.22.0.fits'
infiles = io.list_sweepfiles(src)

# ADM note that the 'main', here, is to distinguish from 'cmx' and 'svX' (with X=1,2,3 etc.)
# ADM for commissioning and SV. The code would default to 'main', anyway, without this keyword.
targets = select_targets(infiles, tcnames=["LRG", "ELG"], survey='main')

# ADM write the targets to file.
io.write_targets(dest, targets, indir=src, survey="main", nside=nside)